Purpose of this notebook is to demonstrate using the `Battery` model to measure forecast quality.

In [1]:
import pandas as pd

dataset = pd.read_csv('./data/forecast_sample.csv', index_col=0, parse_dates=True)
dataset.head()

,Trading Price [$/MWh],Predispatch Forecast [$/MWh]
Timestamp,,
2018-07-01 17:00:00,177.11,97.58039
2018-07-01 17:30:00,135.31,133.10307
2018-07-01 18:00:00,143.21,138.59979
2018-07-01 18:30:00,116.25,128.09559
2018-07-01 19:00:00,99.97,113.29413


A plot I prepared earlier (loaded from `notebooks/data/forecast.png`)

![](./data/forecast.png)

First we dispatch the battery under perfect foresight (if we knew prices ahead of time):

In [2]:
import energypylinear

model = energypylinear.Battery(capacity=1000, power=2, efficiency=1.0)
perfect_foresight = model.optimize(prices=dataset.loc[:, 'Trading Price [$/MWh]'], timestep='30min')

perfect_foresight = pd.DataFrame().from_dict(perfect_foresight)
perfect_foresight.head()

,Import [MW],Export [MW],Gross [MW],Net [MW],Losses [MW],Charge [MWh],Prices [$/MWh],Forecast [$/MWh],Actual [$/30min],Forecast [$/30min]
0,0.0,0.0,0.0,0.0,0.0,0.0,177.11,177.11,0.00,0.00
1,2.0,0.0,2.0,2.0,0.0,0.0,135.31,135.31,135.31,135.31
2,0.0,2.0,-2.0,-2.0,0.0,1.0,143.21,143.21,-143.21,-143.21
3,0.0,0.0,0.0,0.0,0.0,0.0,116.25,116.25,0.00,0.00
4,0.0,0.0,0.0,0.0,0.0,0.0,99.97,99.97,0.00,0.00


We then dispatch the battery using the forecasts:

In [3]:
forecast = model.optimize(
    prices=dataset.loc[:, 'Trading Price [$/MWh]'],
    forecasts=dataset.loc[:, 'Predispatch Forecast [$/MWh]'],
    timestep='30min'
)

forecast = pd.DataFrame().from_dict(forecast)
forecast.head()

,Import [MW],Export [MW],Gross [MW],Net [MW],Losses [MW],Charge [MWh],Prices [$/MWh],Forecast [$/MWh],Actual [$/30min],Forecast [$/30min]
0,2.0,0.0,2.0,2.0,0.0,0.0,177.11,97.58039,177.11,97.58039
1,0.0,0.0,0.0,0.0,0.0,1.0,135.31,133.10307,0.00,0.00000
2,0.0,2.0,-2.0,-2.0,0.0,1.0,143.21,138.59979,-143.21,-138.59979
3,0.0,0.0,0.0,0.0,0.0,0.0,116.25,128.09559,0.00,0.00000
4,2.0,0.0,2.0,2.0,0.0,0.0,99.97,113.29413,99.97,113.29413


Now we can total up the costs for both dispatches under the actual prices, and use this to quantify how much dispatching to the forecast hurt the economics of the battery:

In [4]:
#  we multiply by -1 to convert from a cost to a benefit (the MILP model is a cost minimization problem!)

perfect_total = -1 * perfect_foresight.loc[:, 'Actual [$/30min]'].sum()
forecast_total = -1 * forecast.loc[:, 'Actual [$/30min]'].sum()
forecast_error = perfect_total - forecast_total

print('Optimal dispatch is a benefit of $ {:.2f}'.format(perfect_total))
print('Disptaching under the forecast gave a benefit of $ {:.2f}'.format(forecast_total))
print('')
print('Forecast error is $ {:.2f}'.format(forecast_error))
print('Forecast error is {:.2f} %'.format(100 * forecast_error / perfect_total))

Optimal dispatch is a benefit of $ 429.48
Disptaching under the forecast gave a benefit of $ 302.59

Forecast error is $ 126.89
Forecast error is 29.55 %
